In [ ]:
# add aws later i m p o r t boto3
from azure.storage.blob import BlobServiceClient
from google.cloud import storage
from abc import ABC, abstractmethod

class CloudStorage(ABC):
    @abstractmethod
    def get_file(self, bucket_name: str, blob_name: str) -> bytes:
        pass

# class AwsStorage(CloudStorage):
#     def __init__(self, access_key: str, secret_key: str):
#         self.client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)

#     def get_file(self, bucket_name: str, blob_name: str) -> bytes:
#         response = self.client.get_object(Bucket=bucket_name, Key=blob_name)
#         return response['Body'].read()

class AzureStorage(CloudStorage):
    def __init__(self, connection_string: str):
        self.client = BlobServiceClient.from_connection_string(connection_string)

    def get_file(self, bucket_name: str, blob_name: str) -> bytes:
        container_client = self.client.get_container_client(bucket_name)
        blob_client = container_client.get_blob_client(blob_name)
        return blob_client.download_blob().readall()

class GcpStorage(CloudStorage):
    def __init__(self, credentials_file: str):
        self.client = storage.Client.from_service_account_json(credentials_file)

    def get_file(self, bucket_name: str, blob_name: str) -> bytes:
        bucket = self.client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        return blob.download_as_bytes()

class StorageFactory:
    @staticmethod
    def get_storage(storage_type: str, **kwargs) -> CloudStorage:
        # if storage_type == 'aws':
        #     return AwsStorage(**kwargs)
        elif storage_type == 'azure':
            return AzureStorage(**kwargs)
        elif storage_type == 'gcp':
            return GcpStorage(**kwargs)
        else:
            raise ValueError(f'Invalid storage type: {storage_type}')

storage_type = 'azure' # or 'aws'  or 'gcp'
bucket_name = 'my-bucket'
blob_name = 'file.csv'

# Define the storage factory and create an instance of the appropriate storage implementation
factory = StorageFactory()
storage = factory.get_storage(storage_type, **storage_credentials)


ModuleNotFoundError: No module named 'boto3'